# OpenHSI Ximea Camera Implementations

:::{.callout-tip}

This module can be imported using `from openhsi.cameras import *`

:::

Wrapper class and example code for getting images from the OpenHSI.

:::{.callout-tip}

To use the camera, you will need some calibration files. You can also generate these files following this [guide](https://openhsi.github.io/openhsi/tutorial_calibrate.html) which uses the [`calibrate` module](https://openhsi.github.io/openhsi/calibrate.html).

:::

In [ ]:
#| hide

# documentation extraction for class methods
from nbdev.showdoc import *

# unit tests using test_eq(...)
from fastcore.test import *

# monkey patching class methods using @patch
from fastcore.foundation import *
from fastcore.foundation import patch

# # imitation of Julia's multiple dispatch using @typedispatch
# from fastcore.dispatch import typedispatch

# bring forth **kwargs from an inherited class for documentation
from fastcore.meta import delegates

# external
import numpy as np
import ctypes
import matplotlib.pyplot as plt
import warnings
from tqdm import tqdm
from functools import partial

# internal
from openhsi.capture import OpenHSI
from openhsi.shared import SharedOpenHSI

## XIMEA Camera

Used for the OpenHSI Camera Mark I with a Ximea detetor (with IMX252 sensor, e.g. [MX031CG-SY](https://www.ximea.com/en/products/xilab-application-specific-custom-oem/embedded-vision-and-multi-camera-setup-xix/sony-imx252-fast-color-industrial-camera)). 

Make sure you install the Ximea API beforehand in the instructions https://www.ximea.com/support/wiki/apis/Python 



In [ ]:
#| export cameras

@delegates()
class XimeaCameraBase():
        
    """Core functionality for Ximea cameras"""
    # https://www.ximea.com/support/wiki/apis/Python
    def __init__(self, exposure_ms:float = 10, serial_num:str = None, **kwargs):
        """Initialise Camera"""
        
        super().__init__(**kwargs)
                    
        from ximea import xiapi
        self.xiapi=xiapi # make avalaible for later access just in case.
        
        self.xicam = self.xiapi.Camera()
        
        self.xicam.open_device_by_SN(serial_num) if serial_num else self.xicam.open_device()

        print(f'Connected to device {self.xicam.get_device_sn()}')
        
        self.xicam.enable_horizontal_flip()

        self.xicam.set_binning_vertical(self.settings["binxy"][0])
        self.xicam.set_binning_vertical_mode("XI_BIN_MODE_SUM")

        
        # set window up.
        self.xicam.set_height(self.settings["win_resolution"][0] if self.settings["win_resolution"][0] > 0 else self.xicam.get_height_maximum())
        self.xicam.set_width(self.settings["win_resolution"][1] if self.settings["win_resolution"][1] > 0 else self.xicam.get_width_maximum())
    
        self.xicam.set_offsetY(self.settings["win_offset"][0] if self.settings["win_offset"][0] > 0 else self.xicam.get_offsetY_maximum())
        self.xicam.set_offsetX(self.settings["win_offset"][1] if self.settings["win_offset"][1] > 0 else self.xicam.get_offsetX_maximum())
        

        self.set_exposure(self.settings["exposure_ms"])
        
        self.xicam.set_gain_direct(0.0)

        self.xicam.set_imgdataformat(self.settings["pixel_format"])
        if self.settings["pixel_format"] == "XI_RAW16":
            self.xicam.set_output_bit_depth("XI_BPP_12")
            self.xicam.enable_output_bit_packing()
            
        self.xicam.disable_aeag()
        
        self.rows, self.cols = self.xicam.get_height(), self.xicam.get_width()
        self.img = xiapi.Image()
        
        
    def __exit__(self, *args, **kwargs):
        self.xicam.stop_acquisition()
        self.xicam.close_device()
        
    def set_exposure(self,exposure_ms:float):
            self.xicam.set_exposure_direct(1000*exposure_ms)
            self.settings["exposure_ms"] = self.xicam.get_exposure()/1000  # exposure time rounds, so storing actual value

    def start_cam(self):
        self.xicam.start_acquisition()
    
    def stop_cam(self):
        self.xicam.stop_acquisition()
    
    def get_img(self) -> np.ndarray:
        self.xicam.get_image(self.img)
        return self.img.get_image_data_numpy()
    
    def get_temp(self) -> float:
        return self.xicam.get_temp()

@delegates()
class XimeaCamera(XimeaCameraBase, OpenHSI):
    pass

In [ ]:
#| hardware

def run_ximea():
    with XimeaCamera(n_lines=128, exposure_ms=1, processing_lvl = -1, cal_path="",json_path='../assets/cam_settings_ximea.json') as cam:
        cam.start_cam()
        for i in tqdm(range(cam.n_lines)):
            cam.put(cam.get_img())
        cam.stop_cam()

%prun run_ximea()

#fig = cam.show(robust=True)    
#fig

In [ ]:
#| hardware

with XimeaCamera(n_lines=128, exposure_ms=1, processing_lvl = -1, cal_path="",json_path='../assets/cam_settings_ximea.json') as cam:
    cam.collect()

Allocated 251.56 MB of RAM. There was 12433.93 MB available.


xiAPI: ---- xiOpenDevice API:V4.27.07.00 started ----
xiAPI: EAL_IF_xiFAPI_Top::InitializeDevice sn:CEMAU2105019 name:MC031MG-SY-UB
xiAPI: FGTL_SetParam_to_CAL error from CAL: -1015, addr:x27317e
xiAPI: XiApiToGentlParamModel Auto bandwidth measurement finished (396MBps). Safe limit set to: 317MBps
xiAPI: FGTL_SetParam_to_CAL error from CAL: -10009, addr:x201380
xiAPI: ---- Device opened. Model:MC031MG-SY-UB SN:CEMAU2105019 FwF1:01.31 API:V4.27.07.00 ----


Connected to device b'CEMAU2105019'


xiAPI: xiFAPI_Device::AllocateBuffers Allocating buffers. Count:346 OneBuff:756 KiB All:256 MiB Frm:x10c0047
xiAPI: xiAPI error: Expected XI_OK in:../API/xiFAPI/camera_model/XiApiToGentlParamModel.cpp GetHDR/Line:622
xiAPI: Failed to change thread scheduler, check user limit for realtime priority.
100%|█████████████████████████████████████████████████████████████████████| 128/128 [00:00<00:00, 163.96it/s]
xiAPI: xiFAPI_Device::AcquisitionStop - ignored: acquisition is not running
xiAPI: xiCloseDevice


### Multiprocessing camera export

Export cameras using the SharedOpenHSI class.

In [ ]:
#| export cameras

@delegates()
class SharedXimeaCamera(XimeaCameraBase, SharedOpenHSI):
    pass